# Импорты

In [1]:
import numpy as np
import numpy.ma as ma
import scipy as sp

In [2]:
from numpy.linalg import det, inv
from scipy.linalg import lu 

# Выпуклая оболочка в $n$-мерном пространстве

Чтобы получить тройку, нам необходимо реализовать алгоритм quick hull в $n$-мерном пространстве.

Для начала, без ограничения общности, будем считать, что $n > m$, где $m$ - количество точек. Так как иначе, все точки принадлежат выпуклой оболочке.                           
Этот алгоритм легко обобщаем на произвольную размерность, поэтому, я сначала опишу его для двумерного случая.
Сначала, мы находим прямую (гиперплоскость) (она у нас будет назваться опорной) и разбиваем все оставшиеся точки на $2$ группы - выше нее и ниже нее.
Далее, в каждой группе мы делаем одно и то же с точностью до знака: находим самую высокую точку (это понятие будет введено позже), перебираем каждую точку на опорной прямой (гиперплоскости) и заменяем эту точку на "самую высокую",
и после замены запускаем алгоритм с новой опорной гиперплоскостью и новым множеством точек 
(тем, которое было попало в одну из групп).
Итак, описание алгоритма:

    1. находим опорную гиперплоскость
    2. ххх
    3. ууу
    4. ййй

Разберемся поэтапно. На первом шаге, нам нужно найти опорную гиперплоскость. Это значит найти линейно независимый набор точек в исходном множестве. Это можно сделать с помощью метода Гаусса. Таким образом, если у нас точки лежат в массиве $A$, размера $(n, dim)$, то нам нужно просто применить метод Гаусса для матрицы $A$.
К сожалению, в библиотеках numpy и scipy нет функций для рассчета метода Гаусса напрямую. Но зато есть так называемая $LU$-декомпозиция.

## Определение

$LU$-декомпозиция матрицы $A$ - это разложение вида 
$$\large PA = LU$$
где $P$ - матрица размера $(n, n)$ из нулей и единиц, такая что в каждом столбце и в каждой строке только одна единица. Матрица $P$ отвечает за перестановку строк в матрице $A$. $L$ - нижнетреугольная матрица размера $(n, dim)$, $U$ - верхнетреугольная матрица размера $(dim, dim)$.
Размеры у матриц такие, потому что у нас $n > dim$. В общем случае там $(n, min(n, dim))$ и $(min(n, dim), dim)$ соответственно.

Пожалуйста не пугайтесь того, что вы не знаете линал. Про $LU$-декомпозицию нам достаточно знать то, что написано в определении, и то, что ранг матрицы $A$ равен рангу матрицы $U$.

Для начала докажем несколько вспомогательных утверждений:

## Утверждение

$$\large P^{-1} = P^T$$

### Доказательство

Рассмотрим два случая:
1. $i = j$. Тогда $i$-я строка в $P$ равна $j$-му столбцу в $P^T$. их произведение равно $1$.
2. $i \neq j$. Тогда соответствующие строка и столбец имеют единицы на разных позициях. Следовательно, их произведение равно нулю.

##  Утверждение

Если $rk U < dim$, то все точки лежат внутри одной гиперплоскости. Если $rk U = dim$, то возьмем номера единичных позици первых $dim$ строк матрицы $P$. Тогда строки матрицы $A$ под этими номерами - линейно независимы.

### Доказательство

Пусть $rk U < dim$. $rk U = rk A$, следовательно, $rk A < dim$. Это значит, что любые dim строк в $A$ линейно-зависимы.

Пусть $rk U = dim$. Обозначим $A' = PA$, важно отметить, что $A'$ - это перестановка строк $A$.
Каждая строка $A'$ - это соответствующая строка из $L$, умноженная на $U$. Рассмотрим матрицу $L'$ - как первые $dim$ строк из $L$ и $A''$ - первые $dim$ строк из $A'$. Тогда $A'' = L' U$ - все матрицы квадратные.
Предположим, что $det(A'') = 0$. Тогда $det(A'') = det(L' U) = det(L') det(U)$, но $det(U) \neq 0$, так как $rk U = dim$, а $det(L') \neq 0$ по определению, поэтому $det(A'') \neq 0$.

Теперь, у нас все готово для реализации первого пункта.

In [3]:
def quick_hull(points):
    '''
    :param: points набор точек, для которого необходимо посчитать выпуклую оболочку. 
            должен иметь форму (n, dim), где dim - размерность пространства.
    '''
    n, dim = points.shape
    # считаем LU-декомпозицию. В этой функции возвращаются матрицы p, l, u, 
    # такие, что A = PLU. -> P^T A = LU  
    p, l, u = lu(points, permute_l=False)
    # np.all(u == 0, axis=1) - пробегает по всем строкам u 
    # и возвращает для каждой True если в ней одни нули.
    if np.any(np.all(u == 0, axis=1)):
        # возвращаем исходный набор, так как он принадлежит одной плоскости
        return points
    hyperplane = np.where(p.T[:dim])[1]
    return hyperplane